In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [49]:
# Preparando os dados
original_data = pd.read_excel('Construtoras_sintetico_completo_V2.xlsx')

In [50]:
data = original_data.copy()
data.shape

(21427, 69)

In [34]:
simplified_data = pd.DataFrame()
for compartimento in data['Tipo de Compartimento'].unique():
    temp_data = data[data['Tipo de Compartimento'] == compartimento]
    amostras = temp_data.iloc[:500]  # 150 primeiras amostras de cada compartimento
    simplified_data = pd.concat([simplified_data, amostras], ignore_index=True)

data = simplified_data.copy()
data.shape

(3675, 69)

In [52]:
# Removendo a ocorrência dos valores 'Tanque de Combustível' e 'Sistema de Freio' da coluna 'Tipo de Compartimento'
data = data[~data['Tipo de Compartimento'].isin(['Tanque Combustível Aeronave', 'Sistema de Freio'])].dropna()
data.shape
data['Tipo de Compartimento'].iloc[300:]

302                    Hidráulico
303                   Diferencial
304                       Redutor
305                         motor
306                   Diferencial
                   ...           
21422                       motor
21423                 Transmissão
21424                       motor
21425    Sistema de Arrefecimento
21426                       motor
Name: Tipo de Compartimento, Length: 20945, dtype: object

In [53]:
for column in data.columns:
    if data[column].dtype == np.float64:
        data[column] = data[column].fillna(data[column].mean())
    else:
        data[column] = data[column].fillna(data[column].mode()[0])

# Convertendo modos de falha para reduzir a variabilidade

In [55]:
def group_failures(row):
    if "sem perda de função" in row.lower():
        return row

    else:
        return "Perda das propriedades de maneira a afetar as suas características que permitam uma boa lubrificação"

data["Modos de falha"] = data["Modos de falha"].apply(group_failures)
data["Modos de falha"].value_counts()

Modos de falha
Sem perda de função parcial ou total evidente                                                           12992
Perda das propriedades de maneira a afetar as suas características que permitam uma boa lubrificação     8253
Name: count, dtype: int64

In [56]:
from collections import Counter

# Separar os modos de falha individuais
modos_de_falha = []
for row in data["Modos de falha"].dropna():
    modos_de_falha.extend(row.split(". "))

# Contar as ocorrências de cada modo de falha
contador = Counter(modos_de_falha)

# Criar uma função para substituir os valores na coluna pelo mais frequente
def modo_mais_frequente(texto):
    falhas = texto.split(". ")
    return max(falhas, key=lambda x: contador[x])

# Aplicar a função ao DataFrame
data["Modos de falha"] = data["Modos de falha"].dropna().apply(modo_mais_frequente)

data["Modos de falha"].head()  # Exibir os primeiros resultados

0        Sem perda de função parcial ou total evidente
1        Sem perda de função parcial ou total evidente
2    Perda das propriedades de maneira a afetar as ...
3    Perda das propriedades de maneira a afetar as ...
4        Sem perda de função parcial ou total evidente
Name: Modos de falha, dtype: object

In [57]:
na_list = data.isna().sum()
na_list = na_list[na_list > 0]
print (na_list)

Series([], dtype: int64)


In [58]:
data['S70 µm'] = data['S70 µm'].fillna(data['S70 µm'].mean())
na_list = data.isna().sum()
na_list = na_list[na_list > 0]
print (na_list)

Series([], dtype: int64)


# Label Encoding

In [59]:
from sklearn.preprocessing import LabelBinarizer

# Usando label encoder para transformar as colunas categóricas em numéricas
le = LabelEncoder()

for column in data.select_dtypes(exclude=[np.number]).columns:
  if column != 'Modos de falha':
    data[column] = data[column].astype(str)
    data[column] = le.fit_transform(data[column])
    print(f'Coluna {column} transformada com sucesso')

# Convert y_train to categorical using LabelBinarizer
lb = LabelBinarizer()
data['Modos de falha'] = lb.fit_transform(data['Modos de falha'])
print(f'Coluna Modos de falha transformada com sucesso')

Coluna Equipamento transformada com sucesso
Coluna Tipo de Compartimento transformada com sucesso
Coluna Tipo de Equipamento transformada com sucesso
Coluna Óleo amostrado transformada com sucesso
Coluna Tipo de amostra transformada com sucesso
Coluna Alterações Físico-Químicas transformada com sucesso
Coluna Avaliação do óleo transformada com sucesso
Coluna Desgaste transformada com sucesso
Coluna Avaliação do equipamento transformada com sucesso
Coluna Contaminações transformada com sucesso
Coluna Avaliação de contaminações transformada com sucesso
Coluna Condição da amostra transformada com sucesso
Coluna Modos de falha transformada com sucesso


In [60]:
# Dividindo a base em treino e teste
X = data.drop('Modos de falha', axis=1)
y = data['Modos de falha']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# SMOTE

In [61]:
from imblearn.over_sampling import SMOTE
from collections import Counter

X = data.drop('Tipo de Compartimento', axis=1)
y = data['Tipo de Compartimento']

# Suponha que X são as features e y as classes
print(f"Distribuição original: {Counter(y)}")

# Aplicando SMOTE para gerar mais amostras da classe minoritária
smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)
X_resampled, y_resampled = smote.fit_resample(X, y)

print(f"Nova distribuição: {Counter(y_resampled)}")

# Criando um novo DataFrame com os dados balanceados
data_resampled = pd.DataFrame(X_resampled, columns=X.columns)
data_resampled['Tipo de Compartimento'] = y_resampled

Distribuição original: Counter({13: 8819, 11: 3961, 1: 3608, 2: 2111, 4: 1503, 0: 573, 7: 400, 12: 123, 9: 60, 10: 52, 8: 15, 3: 9, 5: 6, 6: 5})
Nova distribuição: Counter({13: 8819, 1: 8819, 11: 8819, 0: 8819, 4: 8819, 2: 8819, 5: 8819, 7: 8819, 12: 8819, 10: 8819, 9: 8819, 3: 8819, 8: 8819, 6: 8819})


In [62]:
# Dividindo a base em treino e teste
X = data_resampled.drop('Modos de falha', axis=1)
y = data_resampled['Modos de falha']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(48327), np.int64(1): np.int64(38099)}


# Treinamento dos modelos

## Rede Neural

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Definir o modelo MLP (Rede Neural)
mlp_model = MLPClassifier(solver = 'adam', random_state=42)

# Definir os hiperparâmetros para buscar
param_grid_mlp = {
    'hidden_layer_sizes': [(128,), (64,), (32,), (128, 64), (128, 64, 32)],  # Número de neurônios nas camadas ocultas
    'activation': ['relu', 'tanh', 'logistic'],       # Função de ativação
    'alpha': [0.0001, 0.001, 0.01],                  # Regularização (evita overfitting)
    'learning_rate': ['constant', 'adaptive', 'invscaling'],       # Taxa de aprendizado
    'max_iter': [1000, 2000, 5000],
    'batch_size': [32, 64, 128],
}

# Criar o Grid Search com Validação Cruzada
grid_search_mlp = GridSearchCV(mlp_model, param_grid_mlp, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Treinar o modelo
grid_search_mlp.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros para MLP:", grid_search_mlp.best_params_)
print("Melhor acurácia na validação cruzada:", grid_search_mlp.best_score_)

# Avaliar no conjunto de teste
best_mlp_model = grid_search_mlp.best_estimator_
test_accuracy_mlp = best_mlp_model.score(X_test, y_test)
print("Acurácia no conjunto de teste (MLP):", test_accuracy_mlp)

# Comparação visual
y_pred = best_mlp_model.predict(X_test)
comparison = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
print(comparison)

Fitting 5 folds for each of 1215 candidates, totalling 6075 fits

Melhores hiperparâmetros para MLP: {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (64,), 'learning_rate': 'constant', 'max_iter': 1000}
Melhor acurácia na validação cruzada: 0.6812499999999999
Acurácia no conjunto de teste (MLP): 0.6553398058252428
     y_test  y_pred
541       0       0
664       1       0
183       0       0
287       0       0
124       1       0
..      ...     ...
71        1       0
469       0       0
554       0       0
627       0       0
345       1       0

[206 rows x 2 columns]


In [17]:
import time
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import GridSearchCV

start_time_MLP = time.time()

# Definir o modelo MLP (Rede Neural)
mlp_model = MLPClassifier(solver = 'adam', random_state=42)

# Definir os hiperparâmetros para buscar
param_grid_mlp = {
    'hidden_layer_sizes': [(128,), (64,), (32,), (128, 64), (128, 64, 32)],  # Número de neurônios nas camadas ocultas
    'activation': ['relu', 'tanh', 'logistic'],       # Função de ativação
    'alpha': [0.0001, 0.001, 0.01],                  # Regularização (evita overfitting)
    'learning_rate': ['constant', 'adaptive', 'invscaling'],       # Taxa de aprendizado
    'max_iter': [1000, 2000, 5000],
    'batch_size': [32, 64, 128],
}

# Criar o Grid Search com Validação Cruzada
grid_search_mlp = GridSearchCV(mlp_model, param_grid_mlp, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Treinar o modelo
grid_search_mlp.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros para MLP:", grid_search_mlp.best_params_)
print("Melhor acurácia na validação cruzada:", grid_search_mlp.best_score_)

# Avaliar no conjunto de teste
best_mlp_model = grid_search_mlp.best_estimator_
y_pred = best_mlp_model.predict(X_test)
y_prob = best_mlp_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_mlp = best_mlp_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (MLP):", test_accuracy_mlp)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_MLP = time.time()
elapsed_time_MLP = end_time_MLP - start_time_MLP

print(f"Elapsed time: {elapsed_time_MLP:.4f} seconds")

Fitting 5 folds for each of 1215 candidates, totalling 6075 fits

Melhores hiperparâmetros para MLP: {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (64,), 'learning_rate': 'constant', 'max_iter': 1000}
Melhor acurácia na validação cruzada: 0.6812499999999999
Acurácia no conjunto de teste (MLP): 0.6553398058252428
F1-score: 0.05333333333333334
AUC-ROC: 0.5774722932651322
MCC: -0.01770151413524594
Elapsed time: 493.8112 seconds


In [30]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef

start_time_MLP = time.time()

# Definir o modelo MLP (Rede Neural)
mlp_model = MLPClassifier(solver='adam', activation='tanh', alpha=0.0001, batch_size=64, hidden_layer_sizes=(64,), learning_rate='constant', 
                          max_iter=1000, random_state=42)

# Treinar o modelo
mlp_model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = mlp_model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

y_pred = mlp_model.predict(X_test)
y_prob = mlp_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_mlp = mlp_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (MLP):", test_accuracy_mlp)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_MLP = time.time()
elapsed_time_MLP = end_time_MLP - start_time_MLP

print(f"Elapsed time: {elapsed_time_MLP:.4f} seconds")

Acurácia no conjunto de treino: 0.6833333333333333
Acurácia no conjunto de teste (MLP): 0.6553398058252428
F1-score: 0.05333333333333334
AUC-ROC: 0.5774722932651322
MCC: -0.01770151413524594
Elapsed time: 0.1553 seconds


Com a nova base

In [16]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef

start_time_MLP = time.time()

# Definir o modelo MLP (Rede Neural)
mlp_model = MLPClassifier(solver='adam', activation='tanh', alpha=0.0001, batch_size=64, hidden_layer_sizes=(64,), learning_rate='constant', 
                          max_iter=1000, random_state=42)

# Treinar o modelo
mlp_model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = mlp_model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

y_pred = mlp_model.predict(X_test)
y_prob = mlp_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_mlp = mlp_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (MLP):", test_accuracy_mlp)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_MLP = time.time()
elapsed_time_MLP = end_time_MLP - start_time_MLP

print(f"Elapsed time: {elapsed_time_MLP:.4f} seconds")

Acurácia no conjunto de treino: 0.7556109725685786
Acurácia no conjunto de teste (MLP): 0.6878612716763006
F1-score: 0.15625
AUC-ROC: 0.5654934104789456
MCC: 0.056150638436003236
Elapsed time: 0.2856 seconds


In [17]:
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef

start_time_MLP = time.time()

# Definir o modelo MLP (Rede Neural)
mlp_model = MLPClassifier(solver='adam', activation='tanh', alpha=0.0001, batch_size=64, hidden_layer_sizes=(64,), learning_rate='constant', 
                          max_iter=1000, random_state=42)

# Treinar o modelo
mlp_model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = mlp_model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

y_pred = mlp_model.predict(X_test)
y_prob = mlp_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_mlp = mlp_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Isso é dataset completo")

print("Acurácia no conjunto de teste (MLP):", test_accuracy_mlp)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_MLP = time.time()
elapsed_time_MLP = end_time_MLP - start_time_MLP

print(f"Elapsed time: {elapsed_time_MLP:.4f} seconds")

Acurácia no conjunto de treino: 0.6551500705806124
Isso é dataset completo
Acurácia no conjunto de teste (MLP): 0.6508639308855292
F1-score: 0.5212143650499815
AUC-ROC: 0.7074973277390552
MCC: 0.27970187148252046
Elapsed time: 19.6750 seconds


## Floresta Randômica

In [ ]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier()

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000],  # Número de árvores
    'max_depth': [None, 10, 20, 50, 100],    # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10] # Mínimo de amostras para dividir um nó
}

# Criar o Grid Search com validação cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Treinar o modelo com todas as combinações
grid_search.fit(X_train, y_train)

# Mostrar os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar no conjunto de teste
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Acurácia no conjunto de teste:", test_accuracy)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")


Melhores hiperparâmetros: {'max_depth': 100, 'min_samples_split': 5, 'n_estimators': 200}
Melhor acurácia: 0.9979166666666668
Acurácia no conjunto de teste: 0.9902912621359223
Elapsed time: 53.1194 seconds


In [15]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier()

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000],  # Número de árvores
    'max_depth': [None, 10, 20, 50, 100],    # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10] # Mínimo de amostras para dividir um nó
}

# Criar o Grid Search com validação cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Treinar o modelo com todas as combinações
grid_search.fit(X_train, y_train)

# Mostrar os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = best_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")


Melhores hiperparâmetros: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Melhor acurácia: 1.0
Acurácia no conjunto de teste: 0.9854368932038835
F1-score: 0.9784172661870504
AUC-ROC: 0.9997868712702473
MCC: 0.9679493288626012
Elapsed time: 58.6498 seconds


In [31]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

# Avaliar no conjunto de teste
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")

Acurácia no conjunto de treino: 1.0
Acurácia no conjunto de teste: 0.9902912621359223
F1-score: 0.9855072463768116
AUC-ROC: 0.999467178175618
MCC: 0.9784425872172221
Elapsed time: 0.2532 seconds


In [18]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

# Avaliar no conjunto de teste
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")

Acurácia no conjunto de treino: 0.9850374064837906
Acurácia no conjunto de teste: 0.7341040462427746
F1-score: 0.25806451612903225
AUC-ROC: 0.7192221150755383
MCC: 0.2471615909053938
Elapsed time: 0.2111 seconds


In [18]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

# Avaliar no conjunto de teste
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Isso é com dataset completo")

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")

Acurácia no conjunto de treino: 0.7990998079281698
Isso é com dataset completo
Acurácia no conjunto de teste: 0.7676835853131749
F1-score: 0.6752707649345259
AUC-ROC: 0.9061020797175667
MCC: 0.5438171400637811
Elapsed time: 15.9608 seconds


In [32]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(random_state=42)

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000],  # Número de árvores
    'max_depth': [None, 10, 20, 50, 100],    # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10, 20] # Mínimo de amostras para dividir um nó
}

# Criar o Grid Search com validação cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Treinar o modelo com todas as combinações
grid_search.fit(X_train, y_train)

# Mostrar os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = best_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")


Melhores hiperparâmetros: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 500}
Melhor acurácia: 0.7428571428571429
Acurácia no conjunto de teste: 0.773015873015873
F1-score: 0.8098404255319149
AUC-ROC: 0.8409977445950018
MCC: 0.5544575951529928
Elapsed time: 186.5900 seconds


In [47]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(random_state=42)

# Definir os hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000],  # Número de árvores
    'max_depth': [None, 10, 20, 50, 100],    # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10, 20] # Mínimo de amostras para dividir um nó
}

# Criar o Grid Search com validação cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Treinar o modelo com todas as combinações
grid_search.fit(X_train, y_train)

# Mostrar os melhores hiperparâmetros encontrados
print("\nMelhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = best_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")


Melhores hiperparâmetros: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 500}
Melhor acurácia: 0.776734693877551
Acurácia no conjunto de teste: 0.7695238095238095
F1-score: 0.8071713147410359
AUC-ROC: 0.8628399543578115
MCC: 0.5610346028313454
Elapsed time: 298.2967 seconds


In [63]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=500, random_state=42)

model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

# Avaliar no conjunto de teste
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Isso é com dataset completo")

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")

Acurácia no conjunto de treino: 0.990326984935089
Isso é com dataset completo
Acurácia no conjunto de teste: 0.8834233261339093
F1-score: 0.8632419078989042
AUC-ROC: 0.966484261467646
MCC: 0.763224009941382
Elapsed time: 133.5082 seconds


In [68]:
import time
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier

start_time_RF = time.time()

# Definir o modelo
model = RandomForestClassifier(max_depth=50, min_samples_split=5, n_estimators=200, random_state=42)

model.fit(X_train, y_train)

# Avaliar no conjunto de treino
train_accuracy = model.score(X_train, y_train)
print("Acurácia no conjunto de treino:", train_accuracy)

# Avaliar no conjunto de teste
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy = model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Isso é com dataset completo")

print("Acurácia no conjunto de teste:", test_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_RF = time.time()
elapsed_time_RF = end_time_RF - start_time_RF

print(f"Elapsed time: {elapsed_time_RF:.4f} seconds")

Acurácia no conjunto de treino: 1.0
Isso é com dataset completo
Acurácia no conjunto de teste: 0.8990010799136069
F1-score: 0.88545270828868
AUC-ROC: 0.9720745449030677
MCC: 0.7951362142617905
Elapsed time: 61.9265 seconds


## SVM

In [25]:
import time
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.svm import SVC

start_time_SVM = time.time()

# Definir o modelo SVM
svm_model = SVC(C=0.1, kernel='linear', random_state=42, probability=True)

# Treinar o modelo
svm_model.fit(X_train, y_train)

# Avaliar no conjunto de teste
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_svm = svm_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (SVM):", test_accuracy_svm)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_SVM = time.time()
elapsed_time_SVM = end_time_SVM - start_time_SVM

print(f"Elapsed time: {elapsed_time_SVM:.4f} seconds")

Acurácia no conjunto de teste (SVM): 0.7572815533980582
F1-score: 0.5833333333333334
AUC-ROC: 0.816069906223359
MCC: 0.4238216326851869
Elapsed time: 929.0432 seconds


In [26]:
train_accuracy = svm_model.score(X_train, y_train)
print("Acurácia no conjunto de treino (SVM):", train_accuracy)

Acurácia no conjunto de treino (SVM): 0.775


In [17]:
import time
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef
from sklearn.svm import SVC

start_time_SVM = time.time()

# Definir o modelo SVM
svm_model = SVC(C=0.1, kernel='linear', random_state=42, probability=True)

# Treinar o modelo
svm_model.fit(X_train, y_train)

# Avaliar no conjunto de teste
y_pred = svm_model.predict(X_test)
y_prob = svm_model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

test_accuracy_svm = svm_model.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (SVM):", test_accuracy_svm)
train_accuracy = svm_model.score(X_train, y_train)
print("Acurácia no conjunto de treino (SVM):", train_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

end_time_SVM = time.time()
elapsed_time_SVM = end_time_SVM - start_time_SVM

print(f"Elapsed time: {elapsed_time_SVM:.4f} seconds")

Acurácia no conjunto de teste (SVM): 0.6936416184971098
Acurácia no conjunto de treino (SVM): 0.7531172069825436
F1-score: 0.18461538461538463
AUC-ROC: 0.5675024108003857
MCC: 0.08705975222792155
Elapsed time: 1341.6386 seconds


# Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier(
    estimators=[
        ('rf', model),
        ('ann', mlp_model),
        ('svc', svm_model)
    ],
    voting='soft'
)
ensemble.fit(X_train, y_train)
ensemble.predict(X_test)

#printando os resultados
y_pred = ensemble.predict(X_test)

test_accuracy_svm = ensemble.score(X_test, y_test)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)

print("Acurácia no conjunto de teste (ensemble):", test_accuracy_svm)
train_accuracy = ensemble.score(X_train, y_train)
print("Acurácia no conjunto de treino (ensemble):", train_accuracy)
print("F1-score:", f1)
print("AUC-ROC:", auc_roc)
print("MCC:", mcc)

Acurácia no conjunto de teste (ensemble): 0.7167630057803468
Acurácia no conjunto de treino (ensemble): 0.7880299251870324
F1-score: 0.14035087719298245
AUC-ROC: 0.7192221150755383
MCC: 0.15459250714356337


# XGBoost

In [38]:
import time
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_auc_score, matthews_corrcoef

start_time = time.time()

# Classificador base
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Grade de hiperparâmetros expandida
param_grid = {
    'max_depth': [2, 3, 4],                # Reduz profundidade da árvore
    'learning_rate': [0.01, 0.03, 0.05],   # Menores taxas de aprendizado
    'n_estimators': [100, 300, 500],       # Mais árvores para compensar learning_rate menor
    'subsample': [0.6, 0.8, 1],            # Amostragem de dados
    'colsample_bytree': [0.6, 0.8, 1],     # Amostragem de features
    'gamma': [0, 1, 5],                    # Mínimo ganho necessário para fazer split (penaliza splits)
    'reg_alpha': [0, 1, 10],               # Regularização L1 (sparsidade)
    'reg_lambda': [1, 5, 10],              # Regularização L2 (complexidade do modelo)
    'min_child_weight': [1, 3, 5]          # Número mínimo de instâncias em uma folha
}

# Grid Search com F1-score como métrica principal
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=make_scorer(f1_score),
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Treinar grid search
grid_search.fit(X_train, y_train)

# Modelo final
best_model = grid_search.best_estimator_

# Avaliação
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("\nMelhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

print("\nAvaliação no conjunto de teste:")
print("Acurácia (Teste):", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))
print("MCC:", matthews_corrcoef(y_test, y_pred))

end_time = time.time()
print(f"\nTempo total de execução: {end_time - start_time:.2f} segundos")


Fitting 5 folds for each of 19683 candidates, totalling 98415 fits


c:\Users\victo\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:04:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Melhores hiperparâmetros encontrados:
{'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 500, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 1}

Avaliação no conjunto de teste:
Acurácia (Teste): 0.6936416184971098
F1-score: 0.3116883116883117
AUC-ROC: 0.7446158791385407
MCC: 0.15379808171255632

Tempo total de execução: 4034.72 segundos


In [39]:
train_accuracy = best_model.score(X_train, y_train)
print("Acurácia no conjunto de treino (SVM):", train_accuracy)

Acurácia no conjunto de treino (SVM): 1.0


In [40]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef, accuracy_score, make_scorer

start_time = time.time()

# Modelo base
logreg = LogisticRegression(solver='saga', max_iter=5000, random_state=42)

# Grade de hiperparâmetros com foco em regularização
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],     # Tipos de penalização
    'C': [0.01, 0.1, 1, 10],                   # Força da regularização inversa (valores menores = mais regularização)
    'l1_ratio': [0.0, 0.5, 1.0]                # Usado só com elasticnet (entre L1 e L2)
}

# Grid search com F1-score como métrica principal
grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    scoring=make_scorer(f1_score),
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Treinando o modelo
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Avaliação no conjunto de teste
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

print("\nMelhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

print("\nAvaliação no conjunto de teste:")
print("Acurácia (Teste):", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))
print("MCC:", matthews_corrcoef(y_test, y_pred))

end_time = time.time()
print(f"\nTempo total de execução: {end_time - start_time:.2f} segundos")


Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\victo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(



Melhores hiperparâmetros encontrados:
{'C': 0.01, 'l1_ratio': 0.0, 'penalty': 'l2'}

Avaliação no conjunto de teste:
Acurácia (Teste): 0.6994219653179191
F1-score: 0.07142857142857142
AUC-ROC: 0.5388942462230795
MCC: 0.0398049191287005

Tempo total de execução: 54.36 segundos


c:\Users\victo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
